In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.


Import the SDK and some helpers for rendering the output.

In [9]:
from google import genai
from google.genai import types
from google.genai.types import Part

from IPython.display import HTML, Markdown, display,Image

### Set up API key

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

Explore the Gemini model family.

In [4]:
client = genai.Client(api_key=GOOGLE_API_KEY)
for model in client.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01

In [13]:
#select a model
model_id="gemini-2.0-flash-001"

### Few shot prompting

Providing some context or the LLM to understand the task at hand. In this case, we need the output to be in a specific format given some specific data.

In [14]:
few_shot_prompt = """Parse through crowdsourced text, geo-tagged reports on platforms like Ushahidi
that describe what is happening during emergencies, conflicts, elections, or natural disasters. :

EXAMPLE:
Guns shots heard near Toi Market. 3 students caught in the line of fire, ambulances on site.
JSON Response:
```
{
"Type": "gun-violence",
"Location": "Toi Market",
"Impact": "3",
"Response": Ambulances"
}
```

EXAMPLE:
Our house in Uthiru is submerged in flood water. Please help us, a family of six is stranded in the house.
JSON Response:
```
{
"Type": "Floods",
"Location": "Uthiru",
"No of Casualties": "6",
"Response":
}
```


"""

crowdsourced_input = "Mount Etna is quite invisible today. Lots of smoke and tremors near our town in Sicily"

response = client.models.generate_content(
    model=model_id,
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, crowdsourced_input])

print(response.text)

```json
{
"Type": "Volcanic Activity",
"Location": "Sicily",
"Impact": "Smoke and tremors",
"Response": ""
}
```



### Image understanding
If now in the case that users also uploaded images

In [16]:
model_name = "gemini-2.5-pro-exp-03-25"

In [17]:

bounding_box_system_instructions = """
    Return bounding boxes as a JSON array with labels. Never return masks or code fencing. Limit to 25 objects.
    If an object is present multiple times, name them according to their unique characteristic (colors, size, position, unique characteristics, etc..).
      """
     

In [18]:
safety_settings = [
    types.SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="BLOCK_ONLY_HIGH",
    ),
]

In [23]:
import json
import random
import io
from PIL import Image, ImageDraw, ImageFont
from PIL import ImageColor
image_url = "https://raw.githubusercontent.com/LeilaMemoi/Generative_AI/main/chris-gallagher-4zxp5vlmvnI-unsplash.jpg"
im = Image.open(image_url)
im.thumbnail([620,620], Image.Resampling.LANCZOS)
im

FileNotFoundError: [Errno 2] No such file or directory: 'https://raw.githubusercontent.com/LeilaMemoi/Generative_AI/main/chris-gallagher-4zxp5vlmvnI-unsplash.jpg'

In [15]:
image_url = "https://github.com/LeilaMemoi/Generative_AI/blob/main/chris-gallagher-4zxp5vlmvnI-unsplash.jpg"

display(Image(image_url))

image_prompt = """
For humantarian and conflict purposes, you are provided an image and are required to explain what is happening in that regard.
For example, what kind of disaster is in the image

Provide recommendation for the type of response that would be ideal in such a situation.
"""
contents = [
    image_prompt,
    Part.from_uri(file_uri=image_url, mime_type="image/jpeg"),
]
response = client.models.generate_content(model=model_id, contents=contents)
display(Markdown(response.text))

FileNotFoundError: No such file or directory: '





<!DOCTYPE html>
<html
  lang="en"
  
  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"
  data-a11y-animated-images="system" data-a11y-link-underlines="true"
  
  >



  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">
  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>
  <link rel="preconnect" href="https://avatars.githubusercontent.com">

  


  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-74231a1f3bbb.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-8a995f0bacd4.css" /><link data-colo

FileNotFoundError: No such file or directory: '





<!DOCTYPE html>
<html
  lang="en"
  
  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"
  data-a11y-animated-images="system" data-a11y-link-underlines="true"
  
  >



  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">
  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>
  <link rel="preconnect" href="https://avatars.githubusercontent.com">

  


  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-74231a1f3bbb.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-8a995f0bacd4.css" /><link data-colo

<IPython.core.display.Image object>

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Invalid or unsupported file uri: https://github.com/LeilaMemoi/Generative_AI/blob/main/chris-gallagher-4zxp5vlmvnI-unsplash.jpg', 'status': 'INVALID_ARGUMENT'}}